In [78]:
import pandas as pd
import numpy as np
import requests
import json

ALPACA_API_KEY_ID = "_________________________"
ALPACA_API_SECRET_KEY = "_________________________"

ALPACA_API_BASE_URL = "https://data.alpaca.markets/v1"

In [2]:
# List of tickers to grab data for
tickers = [
           "GE", "NIO", "AMC", "EXPR", "XOM",
           "F", "MRO", "ITUB", "BAC", "GME",
           "PBR", "T", "CCL", "VALE", "NOK",
           "NCLH", "RIG", "ABEV", "BBD", "WFC"
]

In [11]:
# Generate Request to pull historical data from alpaca
headers = {"APCA-API-KEY-ID": ALPACA_API_KEY_ID, "APCA-API-SECRET-KEY": ALPACA_API_SECRET_KEY}

url = ALPACA_API_BASE_URL + "/bars/1D"

params = {
    "symbols": ",".join(tickers),
    "start": '2019-01-01T09:30:00-04:00',
    "end": '2021-01-01T09:30:00-04:00',
    "limit": 700
}

response = requests.request("GET", url, headers=headers, params=params)

data = json.loads(response.text)

In [9]:
# Process raw bar data into a dict of DataFrames
symbols_dfs = {}

for symb in tickers:
  symbols_dfs[symb] = pd.DataFrame(data[symb])
  symbols_dfs[symb].set_index(["t"], inplace=True)

symbols_dfs["GE"].head()

,o,h,l,c,v
t,,,,,
1546405200,7.460,8.18,7.410,8.025,118691164
1546491600,8.020,8.20,7.500,8.065,104414212
1546578000,8.190,8.26,7.980,8.210,102709035
1546837200,8.545,8.79,7.390,8.750,162690859
1546923600,8.910,9.04,8.515,8.565,134300920


In [5]:
# Check amount of data for each symbol
for symb in tickers:
  print(symb, "-", len(data[symb]))

GE - 505
NIO - 503
AMC - 505
EXPR - 503
XOM - 505
F - 505
MRO - 505
ITUB - 505
BAC - 505
GME - 505
PBR - 504
T - 505
CCL - 503
VALE - 505
NOK - 505
NCLH - 504
RIG - 504
ABEV - 505
BBD - 505
WFC - 505


In [6]:
# Create raw dataset of potential entry and exit points for a rolling window of a variable amount of days 2 >= d >= 31

# Because we are most likely dealing with limit orders we will want to make sure our orders get filled
# Therefore instead of looking for entry/exit at the lowest/highest values on a day we will buffer them
# by 10% of the bar length for that day

In [73]:
import datetime

# Create an empty dataframe to hold our results
possible_entry_exit = pd.DataFrame(columns=["ticker","entry","exit","profit","days"])

In [74]:
# Raw data generation algorithm
for symb in tickers:
  start_time = datetime.datetime.now()
  for days in range(2,32):
    rows_for_day = []

    for d in range(0, len(data[symb])-days):
      min_value = data[symb][d]['l'] + ((data[symb][d]['h'] - data[symb][d]['l']) * 0.1)
      min_index = d

      secondary_min_value = min_value
      secondary_min_index = min_index

      max_value = max(data[symb][d]['c'], min_value)
      max_index = d

      profit = max_value - min_value

      for t in range(d+1, d + days):
        # if new min and new min/new max profit > last profit, update max to that day's close and update profit
        # elif hold onto new min as secondary incase better max for profit in future
        potential_min = data[symb][t]['l'] + ((data[symb][t]['h'] - data[symb][t]['l']) * 0.1)
        closing = data[symb][t]['c']
        if potential_min < min_value: # if new profit is more update everything
          if (closing - potential_min) > profit:
            min_value = potential_min
            min_index = t

            max_value = closing
            max_index = t

            profit = closing - potential_min
          elif potential_min < secondary_min_value: # elif new profit is not more, update secondary min incase better profit in the future
            secondary_min_value = potential_min
            secondary_min_index = t

        # if new max, just update max and profit
        potential_max = data[symb][t]['h'] - ((data[symb][t]['h'] - data[symb][t]['l']) * 0.1)
        if potential_max > max_value:
          max_value = potential_max
          max_index = t

          if (max_value - secondary_min_value) > profit:
            min_value = secondary_min_value
            min_index = secondary_min_index
            
          profit = max_value - min_value

      row_data = {
          "ticker": symb,
          "entry": min_value,
          "exit": max_value,
          "profit": profit,
          "days": (max_index - min_index)
      }

      rows_for_day.append(row_data)

    possible_entry_exit = pd.concat([possible_entry_exit, pd.DataFrame(rows_for_day)], ignore_index=True)
  
  end_time = datetime.datetime.now()
  process_time = end_time - start_time

  print("Processing %s took %d(s)"%(symb, process_time.seconds))

Processing GE took 0(s)
Processing NIO took 0(s)
Processing AMC took 0(s)
Processing EXPR took 0(s)
Processing XOM took 0(s)
Processing F took 0(s)
Processing MRO took 0(s)
Processing ITUB took 0(s)
Processing BAC took 0(s)
Processing GME took 0(s)
Processing PBR took 0(s)
Processing T took 0(s)
Processing CCL took 0(s)
Processing VALE took 0(s)
Processing NOK took 0(s)
Processing NCLH took 0(s)
Processing RIG took 0(s)
Processing ABEV took 0(s)
Processing BBD took 0(s)
Processing WFC took 1(s)


In [75]:
possible_entry_exit.tail()

,ticker,entry,exit,profit,days
292825,WFC,23.6185,30.3680,6.7495,28
292826,WFC,24.0585,30.3680,6.3095,29
292827,WFC,23.6185,30.4225,6.8040,30
292828,WFC,23.7680,30.4225,6.6545,29
292829,WFC,24.4540,30.4225,5.9685,28


In [76]:
possible_entry_exit.sort_values("profit", ascending=False)

,ticker,entry,exit,profit,days
29225,NIO,21.2500,56.6300,35.380,30
28752,NIO,21.2500,55.1240,33.874,29
29224,NIO,21.6130,55.1240,33.511,30
29226,NIO,24.0530,56.6300,32.577,29
28753,NIO,24.0530,56.6300,32.577,29
...,...,...,...,...,...
146888,PBR,6.9805,6.9805,0.000,0
219903,NCLH,39.5680,39.5680,0.000,0
219914,NCLH,15.2470,15.2470,0.000,0
176156,CCL,15.2310,15.2310,0.000,0


In [77]:
possible_entry_exit.to_csv("COMP542StockGANDataset.csv", index=False)